# Introduction to xarray 

#### presented by Ruth Moorman, Thursday 13th July 2023 (SURF student tutorial) 
#### using ESE101 "Earth's Atmosphere" (2020) recitation by Dave Bonan, Jordan Benjamin, and Tapio Schneider

This tutorial uses output from a state-of-the-art climate model ([CESM2](http://www.cesm.ucar.edu/models/cesm2/)).

In addition to what we cover here I would also recommend this ["xarray in 45 min" tutorial](https://tutorial.xarray.dev/overview/xarray-in-45-min.html), which covers a lot of the basics and which I take material from here, I just wanted to use more visibly Earth-like data for this demonstration. This is all really just to get you started, xarray is **very** well documented and there's load of great resources on their pages so I recommend using them! I would, in particular, recommend these sections on [working with timeseries data](https://docs.xarray.dev/en/stable/user-guide/time-series.html) and [working with weather and climate data](https://docs.xarray.dev/en/stable/user-guide/weather-climate.html), they bring together useful tools nicely. There are also some really useful xarray sections in [Ryan Abernathey's course notes](https://earth-env-data-science.github.io/intro.html).

In [ ]:
## PLOTTING PACKAGES
import matplotlib as mpl        # Python's default plotting package https://matplotlib.org/
import matplotlib.pyplot as plt 
from matplotlib import rc, rcParams
# rc('font', family = 'serif')
# rcParams["text.usetex"]=True
rc('xtick', labelsize=20) 
rc('ytick', labelsize=20) 
rc('axes', labelsize=20)
import cartopy.crs as ccrs      # Mapping  package https://scitools.org.uk/cartopy/docs/latest/

## ANALYSIS PACKAGES
import numpy as np              # Python's numerical and array calculations package https://numpy.org/
import xarray as xr             # Python package for labeled datasets and arrays

In this notebook the key functions we'll cover are:

• `.sel()`

• `.interp()`

• `.groupby()`

• `.weighted()`


• `.where()`

There's a lot more! But these are super handy.

## Loading a "Dataset" (and the "DataArrays", "Coordinates", "Dimensions", and "Attributes" within it ...)

Below, for reference, is a example of what sorts of information can be included in an xarray Dataset as extracted from a [netCDF](https://www.unidata.ucar.edu/software/netcdf/) file type. 


<p align="center">
  <img width="900" height="300" src="https://docs.xarray.dev/en/stable/_images/dataset-diagram.png">
</p>

Let's explore this file structure in our example netCDF file. 



In [ ]:
ds = xr.open_dataset('CESM2_example_data.nc') # load the dataset from a netCDF file
ds                                               # examine the dataset

There's a lot of extra information in here, not just the arrays of raw data, that's kind of the whole point! 

Make sure to click on the icons to the right of each Dataset component, they will display either metadata about that component (square icon) or the data itself (cylindrical icon) if the data is in memory (sometimes xarray does "lazy loading" stuff with where only metadata, not array contents, are kept in memory...but I'm not going into that today!).

If we want to extract or point to a particular component of this Dataset, we use the convention `ds.{thing you want in ds}` or `ds["thing you want in ds"]`, for example, to extract just the `tas` variable (a DataArray structure) we can write:

In [ ]:
ds["tas"]

Note that this doesn't actually show us the values within the array here (part of the lazy loading), so if you want to see what the array within this DataArray structure looks like in a more traditional visualization you can execute `ds["tas"].values()`.

In [ ]:
ds["tas"].values

If you were then to re-run the previous cell you would see a preview of the data within the `tas` array since it's now loaded in memory.

## Data indexing and subseting

**The \#1 perk of xarray is that we get to use readable dimension information (e.g. latitude, longitude, pressure, time values) to index and analyse our data.**

Bascially, we don't need to use classical numerical indexing when working with xarray. Instead of, for example, figuring out that the equator is associated with the 20th index along the 0th axis, we can just ask for data at `lat=0`.

You can find a lot more detail on ways to index [here](https://docs.xarray.dev/en/stable/user-guide/indexing.html), but here's a couple of examples using the `.sel()` terminology (with a cheeky preview of `.interp()`). 



Say that we wanted to pull out surface air temperature (`tas`, here) from the equator, i.e. we want data where `lat` is equal to 0.

Now, if we look at the latitude coordinate of our data (e.g. run `ds["lat"]`) we should notice that there is no `lat` value precisely equal to 0. So If we try to pull out a value with that latitude we will run into an error, execute `ds.tas.sel(lat=0)` if you're curious.


Instead we would need to include a `method` for inexact matches, these are the options:

• `None (default)`: only exact matches

• `pad / ffill`: propagate last valid index value forward

• `backfill / bfill`: propagate next valid index value backward

• `nearest`: use nearest valid index value

In [ ]:
ds.tas.sel(lat=0,method='ffill') # gives us the latitude prior to zero (-0.4712)

In [ ]:
ds.tas.sel(lat=0,method='bfill') # gives us the latitude after zero (0.4712)

Or we could start getting fancy and interpolate our data to the equator using the [`.interp()` function](https://docs.xarray.dev/en/stable/user-guide/interpolation.html).

In [ ]:
ds.tas.interp(lat=0, method='linear')

Another very common action would be to index according to a range, e.g. we may only want Southern Hemisphere data (sensibly, as it is the hemisphere with Australia in it).

In [ ]:
ds.tas.sel(lat=slice(None,0)) # gives data with latitude values between the lower limit (None, could equivalently say -90) and the equator (0)

## Mapping a monthly mean field (where `lat` and `lon` are 1D Dimensions)

Plotting a map of the January temperature field as an example.

In [ ]:
jan_field = ds.tas.sel(time='2010-01').squeeze('time')
jan_field

xarray plays well with all the cartopy mapping functions

In [ ]:
fig = plt.figure(figsize=(15,10))       # initialize a fairly large figure
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global() # show the entire globe
ax.coastlines() # plot coastlines
c = ax.pcolormesh(jan_field.lon,jan_field.lat,jan_field, cmap='RdYlBu_r',transform=ccrs.PlateCarree())
ax.contour(jan_field.lon,jan_field.lat,jan_field,levels=[240,250,260,270,280,300],colors=['k'],alpha=0.2,transform=ccrs.PlateCarree(),vmin=220,vmax=310)
ax.contour(jan_field.lon,jan_field.lat,jan_field,levels=[290],colors=['w'],transform=ccrs.PlateCarree())
ax.set_xticks([-180,-120,-60,0,60,120,180])
ax.set_yticks([-90,-60,-30,0,30,60,90])
ax.set_title('January Mean')
cax,kw = mpl.colorbar.make_axes(ax,location='bottom',pad=0.07,shrink=0.975) # add colorbar axes at bottom
cbar=plt.colorbar(c,cax = cax,orientation='horizontal')
cbar.set_label('Surface Temperature (K)', fontsize = 20)


## Mapping seasonal mean fields using the `.groupby()` function

`.groupby()` is an incredibly useful function within xarray, there's a detailed overview [here](https://docs.xarray.dev/en/stable/user-guide/groupby.html) but really this is a very flexible function. `.resample()`, `.rolling()` and `.coarsen()` are useful related functions I won't go into here but recommend you checking out. Here we will use `.groupby()` to compute seasonal mean `tas` fields, but essentially it can be used for any situation where you want to bin or cluster data in groups prior to executing a function.

In [ ]:
tas_seasons = ds.tas.groupby('time.season').mean() # an example of cool time functionality in xarray
tas_seasons

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(25,14), subplot_kw={'projection': ccrs.PlateCarree()})
for i, season in enumerate(('DJF', 'MAM', 'JJA', 'SON')):
    seas_data = tas_seasons.sel(season=season) # select the season we want
    ax=axes.flatten()[i]                       # specify the axis
    ax.set_title(season, fontsize=20)
    ax.set_global() # show the entire globe
    ax.coastlines() # plot coastlines
    c = ax.pcolormesh(seas_data.lon,seas_data.lat,seas_data, cmap='RdYlBu_r',transform=ccrs.PlateCarree(), vmin=220,vmax=310)
    ax.contour(seas_data.lon,seas_data.lat,seas_data,levels=[240,250,260,270,280,300],colors=['k'],alpha=0.2,transform=ccrs.PlateCarree())
    ax.contour(seas_data.lon,seas_data.lat,seas_data,levels=[290],colors=['w'],transform=ccrs.PlateCarree())
    ax.set_xticks([-180,-120,-60,0,60,120,180])
    ax.set_yticks([-90,-60,-30,0,30,60,90])
    if i in (0,2):
        ax.set_ylabel('Latitude')
    if i in (2,3):
        ax.set_xlabel('Longitude')
cax = fig.add_axes([0.92, 0.14, 0.017, 0.72])
cbar=plt.colorbar(c,cax = cax,orientation='vertical')
cbar.set_label('Surface Temperature (K)', fontsize = 20)
plt.show()

## Weighted spatial means using `.weighted()`

Say we want to look at a global mean vertical profile of temperature using the `tas` variable. Because the earth is spherical, data points as high latitudes describe less area than data points at low latitudes. The proper latitude ($\phi$) weighting is:
$$\text{weight} = \cos(\phi) $$
If we do not weight correctly, we may overreppresent high latitude areas that are cold but take up little area, decreasing the global-mean temperature. See [this example](http://xarray.pydata.org/en/stable/examples/area_weighted_temperature.html).

Xarray can perform the weighting nearly automatically with the [`.weighted()` function](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.weighted.html):

In [ ]:
weight = np.cos(np.deg2rad(ds.ta['lat']))    # convert degrees to radians and calculate the weights using the full xarray lat object. Using the full ojbect and all its metadata is impoprtant!

In [ ]:
temperature_weighted = ds['ta'].weighted(weight)                          # calculate a weighted mean object in xarray ()
plot_data  = temperature_weighted.mean(['lat','lon','time']).transpose()  # take the mean over latitude, longitude, and time leaving only a vertical profile

We could do this manually as well, the definition of a weighted mean is:
$$ \left<x\right>_w = \frac{\sum w_i x_i}{\sum w_i} = \sum x_i \frac{ w_i }{\sum w_i}$$
so we would need to calculate the following:
$$ \left<x\right>_w = x_i \frac{ w_i }{\sum w_i} $$
and sum over latitude. <br>

One might be tempted to do the following:

In [ ]:
wi_sumwi = weight/weight.sum('lat')                                                 # compute the normalised weights term above
plot_data_1   = (ds['ta'] * wi_sumwi).sum(['lat']).mean(['lon','time']).transpose() # for latitude, we only have the sum as described above, and then a mean over lat and time to leave a vertical profile

But this is actually incorrect as it includes weights for data points where there is no temperature values recorded (when the surface pressure is higher than the lower limit of the y-axis). If we wanted to do this manually (i.e. not using the `.weighted()` function) we would need to remove weights where temperature data does not exist. This can be done by, for example, expanding the weights array to 2 dimensions and masking out wherever no `ta` data is available. 

In [ ]:
weight_masked = ds['ta']*0+weight                        # note ds["ta"]*0 is a mask of 0 where ta data exists and NaN where ta data is absent
wi_sumwi_masked = weight_masked/weight_masked.sum('lat') # compute the normalised weights term above excluding regions with no data
plot_data_2   = (ds['ta'] * wi_sumwi_masked).sum(['lat']).mean(['lon','time']).transpose() # for latitude, we only have the sum as described above, and then a mean over lat and time to leave a vertical profile

And we can compare these options to an unweighted mean.

In [ ]:
plot_data_3   = ds["ta"].mean(['lat','lon','time']).transpose() # simple mean (incorrect)

In [ ]:
fig = plt.figure(figsize=(8, 12)) # assign a vertically tall figure
ax  = plt.axes()                  # assign axes to the figure

plot_data.plot(  ax=ax, y='plev',yscale='log',ylim=[plot_data.plev.max()  ,plot_data.plev.min()  ],linewidth=10,              color='cyan' , label='xarray .weighted() function'  ) # plot profile, log scale, width 10, color cyan,  label as xarray
plot_data_1.plot(ax=ax, y='plev',yscale='log',ylim=[plot_data_2.plev.max(),plot_data_2.plev.min()],linestyle='--',             color='black', label='manual incorrect weighting (missing data)') # plot profile, log scale,           color black, label as method 1
plot_data_2.plot(ax=ax, y='plev',yscale='log',ylim=[plot_data_2.plev.max(),plot_data_2.plev.min()],linestyle='--', linewidth=3 ,color='red', label='manual correct weighting') # plot profile, log scale,           color black, label as method 1
plot_data_3.plot(ax=ax, y='plev',yscale='log',ylim=[plot_data_2.plev.max(),plot_data_2.plev.min()],                           color='black', label='unweighted mean') # plot profile, log scale,           color black, label as method 1
ax.set_xlabel('Temperature (K)');ax.set_ylabel('Pressure (Pa)')
ax.legend(fontsize=15) # you see the problems at the bottom from some missing data
plt.show()

We can also check if weighting by the length of each month makes a difference...

In [ ]:
weight_time = ds['time.days_in_month'] # another example of cool time functionality
weight_time

In [ ]:
temperature_spatial_weighted = ds['ta'].weighted(weight)                        # calculate a weighted mean object in xarray ()
temperature_spatial_weighted_mean  = temperature_spatial_weighted.mean(['lat','lon']) 
temperature_time_weighted = temperature_spatial_weighted_mean.weighted(weight_time)
plot_data_4  = temperature_time_weighted.mean(['time']).transpose()  # take the mean over latitude, longitude, and time leaving only a vertical profile

In [ ]:
(plot_data_4 - plot_data).values # comparatively very small errors, may matter in some cases though!

## Masking using the `.where()` function

The xarray [`.where()` function](https://docs.xarray.dev/en/stable/generated/xarray.where.html) is another useful one and it's use comes up in a couple of contexts. A couple that come to mind include (i) creating a mask based on some critereon, (ii) removing outliers from a dataset, and (iii) selecting a subset from your dataset based off a coordinate that is not a dimension (e.g. if you have irregular 2D latitude and longitude fields but want to subset by latitude).

As a simple example here, let's just repeat our previous map plot but exclude temperature values below 290 K (white contour previously).

In [ ]:
fig = plt.figure(figsize=(15,10))       # initialize a fairly large figure
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global() # show the entire globe
ax.coastlines() # plot coastlines

## change is here
c = ax.pcolormesh(jan_field.lon,jan_field.lat,jan_field.where(jan_field>290), cmap='RdYlBu_r',transform=ccrs.PlateCarree(),vmin=220,vmax=310) 

ax.contour(jan_field.lon,jan_field.lat,jan_field,levels=[240,250,260,270,280,300],colors=['k'],alpha=0.2,transform=ccrs.PlateCarree())
ax.set_xticks([-180,-120,-60,0,60,120,180])
ax.set_yticks([-90,-60,-30,0,30,60,90])
ax.set_title('January Mean')
cax,kw = mpl.colorbar.make_axes(ax,location='bottom',pad=0.07,shrink=0.975) # add colorbar axes at bottom
cbar=plt.colorbar(c,cax = cax,orientation='horizontal')
cbar.set_label('Surface Temperature (K)', fontsize = 20)


## Mapping a field (where `lat` and `lon` are 2D Coordinates)

**a brief interlude with another data file**

You may run into circumstances where the spatial information of your data are actually 2 dimensional themselves! Many general circulation models have irregular (w.r.t latitude and longitude) grids and yet you may still want to plot maps with latitude and longtiude axes (or do other operations along these coordinates). Here's a quick example of how you can use 2 dimensional coordinate data, using some bathymetry from a regional ocean model I personally use.

In [ ]:
bathy = xr.open_dataset('bathy.nc')
bathy

In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(12,10))
# bathy shading
c = ax.pcolormesh(-bathy.bathy, cmap='viridis_r', vmax=6000, vmin=0)
# bathy contours
ax.contour(-bathy.bathy, levels = [1000,2000,3000,4000,5000], colors=['white'], linewidths=0.6)
ax.contour(-bathy.bathy, levels = [500,750], colors=['grey'], linewidths=0.6)

# axis labels
ax.set_xlabel('XC (model grid)')
ax.set_ylabel('YC (model grid)')
cax = fig.add_axes([0.92, 0.125, 0.02, 0.755])
cbar=plt.colorbar(c,cax = cax,orientation='vertical')
cbar.set_label('Bathymetry (m)', fontsize = 20)

plt.show()

In [ ]:
fig, (ax) = plt.subplots(1, 1, figsize=(12,10))

# bathy shading
c = ax.pcolormesh(bathy.LONC, bathy.LATC,-bathy.bathy, cmap='viridis_r', vmax=6000, vmin=0, shading='gouraud')
# # bathy contours
ax.contour(bathy.LONC, bathy.LATC,-bathy.bathy, levels = [1000,2000,3000,4000,5000], colors=['white'], linewidths=0.6)
ax.contour(bathy.LONC, bathy.LATC,-bathy.bathy, levels = [500,750], colors=['grey'], linewidths=0.6)

# axis labels
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
cax = fig.add_axes([0.92, 0.125, 0.02, 0.755])
cbar=plt.colorbar(c,cax = cax,orientation='vertical')
cbar.set_label('Bathymetry (m)', fontsize = 20)

plt.show()